<a href="https://colab.research.google.com/github/bewithankit/CS3DP19/blob/main/word_count_using_apache_spark_with_pyspark_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Set Up the Environment:**


*   First, you need to install all the necessary components to run Spark in Colab. This includes Java, Spark itself, and findspark, a Python library that makes it easier to find Spark.

*   Use the `!` operator to run the following commands in a Colab cell to install Java and Spark, and set up the environment variables:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


*   Then, set the environment variables for Java and Spark in Python:



In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"


**2. Initialize Spark:**


*   Import `findspark` and initialize it. Then create a SparkContext.




In [3]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('wordCount')
sc = SparkContext(conf=conf)
# sc = SparkContext.getOrCreate(conf=conf)


**3. Create an RDD:**

* Create a base RDD from a simple list of words and parallelize it.

In [4]:
wordsList = ['blue', 'orange', 'red', 'blue', 'red', 'blue']
wordsRDD = sc.parallelize(wordsList, 5)
print(type(wordsRDD))  # Should print <class 'pyspark.rdd.RDD'>

# Use glom to see how the data is distributed across partitions
glommedRDD = wordsRDD.glom()
print(glommedRDD.collect())

<class 'pyspark.rdd.RDD'>


**4. Transform the RDD:**

* Use a map operation to transform each word in the RDD.

In [5]:
# Using a defined function
def webify(x):
    return x+".com"
webified_RDD = wordsRDD.map(webify)
print(webified_RDD.collect())

# Using a lambda function # lambda <args>: <method body>
webified_RDD = wordsRDD.map(lambda word: word + '.com')
print(webified_RDD.collect())


['blue.com', 'orange.com', 'red.com', 'blue.com', 'red.com', 'blue.com']
['blue.com', 'orange.com', 'red.com', 'blue.com', 'red.com', 'blue.com']


**5. Create Pair RDDs:**

* Map each word to a tuple containing the word and the number 1, to prepare for counting.

In [6]:
wordPairs = wordsRDD.map(lambda word: (word, 1))
print(wordPairs.collect())


[('blue', 1), ('orange', 1), ('red', 1), ('blue', 1), ('red', 1), ('blue', 1)]


**6. Count the Words:**

* Two approaches can be used: `groupByKey()` or `reduceByKey()`. The latter is more efficient.

In [9]:
# Using groupByKey()
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
  print ('{0}: {1}'.format(key, list(value)))

wordCountsGrouped = wordsGrouped.mapValues(sum)
print(wordCountsGrouped.collect())

# Using reduceByKey()
wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
print("\n")
print(wordCounts.collect())


orange: [1]
red: [1, 1]
blue: [1, 1, 1]
[('orange', 1), ('red', 2), ('blue', 3)]


[('orange', 1), ('red', 2), ('blue', 3)]


**7. Run the Complete Application:**

* Combine all the steps into a single sequence to count the words.

In [10]:
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(lambda x, y: x + y)
                       .collect())
print(wordCountsCollected)


[('orange', 1), ('red', 2), ('blue', 3)]


**8. Shutting Down:**

* After you are done, stop the SparkContext to free up resources.

In [11]:
sc.stop()
